
# Sync Batch Normalization (SyncBatchNorm) — Baştan Sona

> Bu notebook, **Sync Batch Normalization** kavramını **temelden ileri düzeye** kadar anlatır:  
> - Neden ihtiyaç duyulur?  
> - **BatchNorm / LayerNorm / GroupNorm** ile farkları  
> - Çoklu GPU/çoklu süreç (DDP) senaryolarında davranış  
> - PyTorch’ta doğru kurulum, dönüşüm (`convert_sync_batchnorm`)  
> - Performans/maliyet, pratik tuzaklar, “ne zaman kullanmalı?” karar çerçevesi  
> - CNN/Detection/Segmentation örnek kullanım kalıpları  

---

## İçindekiler
1. Batch Normalization temeli (kısa ama sağlam)  
2. SyncBatchNorm nedir? Çalışma mantığı  
3. Ne zaman gerekli olur? (küçük batch / çoklu GPU)  
4. Diğer normalization türleriyle karşılaştırma (BN, LN, GN, IN)  
5. PyTorch’ta SyncBatchNorm:  
   - DDP ile doğru kullanım  
   - Modeli SyncBN’e çevirme  
   - Sık hatalar (DataParallel, tek GPU, eval modu vb.)  
6. Performans ve iletişim maliyeti (All-Reduce)  
7. Uygulama alanları (Detection/Segmentation/Video/CV)  
8. Karar rehberi + pratik checklist  
9. Mini örnek: BN vs SyncBN (kod şablonları)

> Not: Kodlar “çalıştırılabilir şablon” olarak verilir. Çoklu GPU gerektiren hücreler, tek GPU/CPU ortamında **bilgilendirici** amaçlıdır.



## 1) Batch Normalization (BN) Temeli

### 1.1. BN neyi çözer?
Derin ağlarda katmanların aktivasyon dağılımları eğitim boyunca kayar (internal covariate shift olarak popülerleşti). BN:
- Aktivasyonları **mini-batch** istatistikleriyle normalize eder.
- Daha stabil gradyan, daha yüksek öğrenme oranı, daha hızlı yakınsama sağlar.
- Düzenlileştirici (regularizer) etkisi de vardır (mini-batch gürültüsü).

### 1.2. BN matematiği (özet)
Bir kanala (feature channel) ait aktivasyonlar için, mini-batch üzerinde:
- Ortalama:  \(\mu_B = \frac{1}{m}\sum_{i=1}^{m} x_i\)
- Varyans:  \(\sigma_B^2 = \frac{1}{m}\sum_{i=1}^{m}(x_i-\mu_B)^2\)

Normalize:
\[
\hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}
\]

Sonra öğrenilebilir affine parametrelerle ölçek/kaydır:
\[
y_i = \gamma \hat{x}_i + \beta
\]

### 1.3. Train vs Eval farkı
- **Train:** \(\mu_B, \sigma_B^2\) mini-batch’ten hesaplanır.
- **Eval:** eğitim boyunca biriken **running_mean** ve **running_var** kullanılır.

**Kritik nokta:** BN’in kalitesi, özellikle **batch istatistiklerinin güvenilirliğine** bağlıdır. Batch çok küçükse istatistikler “gürültülü” olur.



## 2) SyncBatchNorm Nedir?

### 2.1. Problem: Çoklu GPU + küçük per-GPU batch
Distributed Data Parallel (DDP) eğitimde tipik olarak:
- Her GPU bir “process” çalıştırır.
- Her process kendi mini-batch’ini işler (ör. GPU başına batch=2).
- BN istatistikleri **lokal mini-batch’ten** hesaplanır.

Eğer GPU başına batch küçükse (1–4 gibi), BN istatistikleri kararsızlaşır ⇒ performans düşer.

### 2.2. Çözüm: SyncBatchNorm
**SyncBatchNorm**, BN istatistiklerini **tüm süreçler/GPU’lar arasında senkronize eder**.

Yani, her GPU kendi aktivasyonlarından kısmi istatistik çıkarır, sonra:
- Tüm GPU’lar arasında **all-reduce** ile toplanır.
- Global ortalama/varyans hesaplanır.
- Her GPU aynı \(\mu\) ve \(\sigma^2\) ile normalize eder.

Bu sayede “global batch” sanki tek GPU’da daha büyük batch ile yapılmış gibi davranır.

### 2.3. Basit sezgi
- Normal BN: “Her GPU kendi küçük örneğine bakıp karar veriyor.”  
- SyncBN: “Tüm GPU’lar konuşup ortak karar veriyor.” ✅

Ama konuşmanın maliyeti var: **iletişim (communication) overhead**.



## 3) Ne Zaman Gerekli Olur?

### 3.1. Tipik ihtiyaç senaryoları
- **Detection / Segmentation**: yüksek çözünürlük → GPU belleği sınırı → batch küçülür.
- **3D / Video**: tensörler büyük → batch küçülür.
- Büyük backbone + ağır head (Mask R-CNN, RetinaNet, U-Net varyantları, vb.)
- “Global batch” hedefleniyor ama “per-GPU batch” küçük kalıyor.

### 3.2. Ne zaman gereksiz ya da zararlı olabilir?
- **Per-GPU batch zaten yeterince büyükse** (kabaca 16+ gibi; model ve veri türüne göre değişir): SyncBN çoğu zaman **fazladan overhead**.
- **Tek GPU** eğitimde: SyncBN’in anlamı yok (senkronize edilecek başka süreç yok).
- Çok yüksek iletişim maliyeti olan sistemlerde (yavaş interconnect): throughput düşebilir.

### 3.3. “Küçük batch” için alternatifler
SyncBN tek çözüm değil:
- **GroupNorm (GN):** batch boyutundan bağımsız, özellikle detection’da sık tercih edilir.
- **LayerNorm (LN):** transformer’larda default.
- **Batch size büyütme:** gradient accumulation ile efektif batch artar ama BN istatistiği hâlâ “micro-batch” üzerinden hesaplanır (yani BN sorununu doğrudan çözmez).



## 4) Diğer Normalization Türleriyle Farkları

Aşağıda kısa ama “karar vermeye yarayan” bir karşılaştırma var.

### 4.1. BatchNorm (BN)
- Normalize ekseni: batch + (H,W) (Conv için kanal bazlı)
- **Batch boyutuna duyarlı**
- CNN’lerde çok başarılı (özellikle sınıflandırma)

### 4.2. SyncBatchNorm (SyncBN)
- BN’in aynısı ama batch istatistiklerini **tüm GPU’lardan** toplar
- DDP çoklu GPU senaryolarında küçük batch sorununu azaltır
- **İletişim maliyeti** var

### 4.3. LayerNorm (LN)
- Normalize ekseni: örnek içindeki feature boyutu (token/kanal)
- Batch boyutundan bağımsız
- Transformer mimarilerinde standart

### 4.4. GroupNorm (GN)
- Kanalları gruplara bölüp normalize eder
- Batch boyutundan bağımsız
- Detection/Segmentation’da çok kullanılır (özellikle küçük batch)

### 4.5. InstanceNorm (IN)
- Her örneği (ve kanalı) kendi içinde normalize eder
- Stil transfer gibi işlerde yaygın

> Özet karar:  
> - CNN + **küçük per-GPU batch** + **çoklu GPU** ⇒ SyncBN mantıklı.  
> - CNN + küçük batch ama iletişim pahalı ⇒ GN daha stabil olabilir.  
> - Transformer ⇒ LN çoğu zaman zaten yeterli.


In [2]:
# Kısa bir tabloyu yazdırmak için: (not: bu hücre opsiyonel)
import pandas as pd

df = pd.DataFrame([
    ["BatchNorm", "Batch + (H,W)", "Evet", "Yok", "CNN sınıflandırma"],
    ["SyncBatchNorm", "Global Batch + (H,W)", "Evet (global)", "Var", "Detection/Segmentation (DDP)"],
    ["LayerNorm", "Feature/Channel", "Hayır", "Yok", "Transformer"],
    ["GroupNorm", "Channel groups", "Hayır", "Yok", "Detection/Segmentation"],
    ["InstanceNorm", "Per-sample, per-channel", "Hayır", "Yok", "Style transfer"],
], columns=["Norm", "Normalize ekseni", "Batch'e duyarlı?", "İletişim maliyeti", "Tipik kullanım"])

df

,Norm,Normalize ekseni,Batch'e duyarlı?,İletişim maliyeti,Tipik kullanım
0,BatchNorm,"Batch + (H,W)",Evet,Yok,CNN sınıflandırma
1,SyncBatchNorm,"Global Batch + (H,W)",Evet (global),Var,Detection/Segmentation (DDP)
2,LayerNorm,Feature/Channel,Hayır,Yok,Transformer
3,GroupNorm,Channel groups,Hayır,Yok,Detection/Segmentation
4,InstanceNorm,"Per-sample, per-channel",Hayır,Yok,Style transfer



## 5) PyTorch’ta SyncBatchNorm

### 5.1. Altın kural: SyncBN **DDP ile** anlamlıdır

**DDP = aynı modelin birden fazla kopyasını, birden fazla GPU’da, senkronize şekilde eğitme mekanizmasıdır.Model tekmiş gibi davranır ama iş yükü GPU’lara bölünmüştür 🔁🔥,**

PyTorch’ta SyncBatchNorm çoğunlukla şu senaryoda doğru çalışır:
- `torch.nn.parallel.DistributedDataParallel (DDP)`
- Her GPU = ayrı process
- NCCL backend (GPU için)

**DataParallel** (tek process, çok GPU) genelde önerilmez; SyncBN için de doğru/optimum yol değildir.

### 5.2. Modeli SyncBN’e çevirme
PyTorch’ta yaygın pratik:
1) Modeli normal BN ile kur (nn.BatchNorm2d vs.)  
2) DDP’ye sarmalamadan önce:  
   `model = torch.nn.SyncBatchNorm.convert_sync_batchnorm(model)`  
3) Sonra DDP ile wrap et.

### 5.3. Train/Eval davranışı
- Train: istatistikler sync edilir.
- Eval: running stats kullanılır (senkronizasyon yok).

### 5.4. Dikkat: BatchNorm “running stats”
SyncBN de BN gibi `running_mean`, `running_var` tutar. Eğitim sırasında global istatistiklerle güncellenir.

### 5.5. Sık tuzaklar
- **DDP olmadan SyncBN kullanmak:** pratikte fayda yok/yanlış beklenti.
- **Batch çok küçük + BN (sync yok):** mAP/IoU/accuracy düşebilir.
- `model.train()` / `model.eval()` modlarını yanlış yönetmek.
- `find_unused_parameters=True` gibi DDP ayarları ile gereksiz overhead.


In [3]:
# PyTorch'ta convert_sync_batchnorm örneği (şablon)
import torch
import torch.nn as nn

class TinyCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.net(x)

model = TinyCNN()
print("Önce:", model)

model_sync = nn.SyncBatchNorm.convert_sync_batchnorm(model)
print("\nSonra:", model_sync)


Önce: TinyCNN(
  (net): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
)

Sonra: TinyCNN(
  (net): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): SyncBatchNorm(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): SyncBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
  )
)



## 6) DDP ile “Doğru” Eğitim İskele (Skeleton)

Aşağıdaki iskelet, SyncBN kullanımı için en temiz yoldur:

1. `torchrun --nproc_per_node=<GPU_SAYISI> train.py ...`
2. `init_process_group(backend="nccl")`
3. `torch.cuda.set_device(local_rank)`
4. Modeli GPU’ya taşı
5. `convert_sync_batchnorm`
6. DDP ile sar: `DDP(model, device_ids=[local_rank])`
7. DistributedSampler kullan
8. Eğitim döngüsü

> Aşağıdaki kod hücresi “tek notebook içinde” örnek iskelet veriyor.  
> Bunu genelde `train.py` şeklinde script’e taşımak daha doğru (DDP notebook içinde uğraştırır).


In [7]:

# DDP + SyncBN eğitim iskeleti (script şablonu)
# Bunu tipik olarak train.py içine koyarsın. Notebook içinde "doğrudan çalıştırma" hedeflenmemiştir.

ddp_syncbn_template = r'''
import os
import torch
import torch.nn as nn
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, DistributedSampler
from torchvision import datasets, transforms

def setup():
    dist.init_process_group(backend="nccl")
    local_rank = int(os.environ["LOCAL_RANK"])
    torch.cuda.set_device(local_rank)
    return local_rank

def cleanup():
    dist.destroy_process_group()

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(64, 10),
        )
    def forward(self, x): return self.net(x)

def main():
    local_rank = setup()

    # dataset
    tfm = transforms.Compose([transforms.Resize((64,64)), transforms.ToTensor()])
    ds = datasets.FakeData(size=2000, image_size=(3,64,64), num_classes=10, transform=tfm)
    sampler = DistributedSampler(ds, shuffle=True)
    dl = DataLoader(ds, batch_size=8, sampler=sampler, num_workers=2, pin_memory=True)

    model = Model().cuda(local_rank)

    # 1) SyncBN dönüşümü (DDP'den ÖNCE)
    model = nn.SyncBatchNorm.convert_sync_batchnorm(model)

    # 2) DDP wrap
    model = DDP(model, device_ids=[local_rank])

    opt = torch.optim.AdamW(model.parameters(), lr=1e-3)
    loss_fn = nn.CrossEntropyLoss()

    model.train()
    for epoch in range(5):
        sampler.set_epoch(epoch)
        for x, y in dl:
            x = x.cuda(local_rank, non_blocking=True)
            y = y.cuda(local_rank, non_blocking=True)
            opt.zero_grad(set_to_none=True)
            logits = model(x)
            loss = loss_fn(logits, y)
            loss.backward()
            opt.step()

        if local_rank == 0:
            print(f"epoch {epoch} done")

    cleanup()

if __name__ == "__main__":
    main()
'''

print(ddp_syncbn_template[:1200] + "\n...\n" + ddp_syncbn_template[-400:])



import os
import torch
import torch.nn as nn
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader, DistributedSampler
from torchvision import datasets, transforms

def setup():
    dist.init_process_group(backend="nccl")
    local_rank = int(os.environ["LOCAL_RANK"])
    torch.cuda.set_device(local_rank)
    return local_rank

def cleanup():
    dist.destroy_process_group()

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.AdaptiveAvgPool2d(1),
            nn.Flatten(),
            nn.Linear(64, 10),
        )
    def forward(self, x): return self.net(x)

def main():
    local_rank = setup()

    # dataset
    tfm = transforms.Compose([


### Bu script nasıl çalıştırılır?
Örneğin 2 GPU için:
```bash
torchrun --nproc_per_node=2 train.py
```
- `torchrun` DDP süreçlerini başlatır.
- `LOCAL_RANK` gibi env değişkenleri otomatik gelir.

> Eğer tek GPU/CPU ortamındaysan, SyncBN’e geçmek **anlamsız** olur. Normal BN veya GN seç.



## 7) Performans ve İletişim Maliyeti

SyncBN’in bedeli: **All-Reduce** iletişimi.

### 7.1. Neden yavaşlatır?
Her SyncBN katmanı, forward aşamasında (istatistik için) ve bazen backward aşamasında iletişim gerektirir.
- Modelde çok BN varsa (ResNet gibi), iletişim sayısı artar.
- Interconnect yavaşsa (PCIe vs NVLink vs InfiniBand), fark büyür.

### 7.2. Pratik optimizasyonlar
- SyncBN’i sadece kritik bloklarda kullanmak (her yerde şart değil)
- Alternatif: GN (communication yok)
- Daha büyük per-GPU batch mümkünse yapmak
- Mixed precision (AMP) compute’ı hızlandırır ama iletişim overhead’i aynı kalabilir

### 7.3. “Doğru” beklenti
SyncBN genelde:
- **Stabilite/kalite** kazandırır (özellikle küçük per-GPU batch’te)
- Ama **throughput** düşebilir

Bu yüzden hedef metrik (mAP, IoU, accuracy) vs “time-to-train” dengesi kurulur.



## 8) Uygulama Alanları ve Tipik Kullanım

### 8.1. Object Detection
- YOLO, Faster R-CNN, RetinaNet, DETR dışındaki klasik CNN head’li modeller
- High-res input → küçük batch → BN bozulur
- SyncBN veya GN tercih edilir

### 8.2. Semantic / Instance Segmentation
- U-Net türevleri, DeepLab, Mask R-CNN
- Batch küçülür, SyncBN yaygın

### 8.3. Video / 3D
- 3D Conv’lar bellek yer → batch küçülür
- SyncBN ile stabilizasyon

### 8.4. Büyük ölçekli sınıflandırma
Eğer global batch zaten büyükse SyncBN şart değil.
Ama “çok GPU + per-GPU batch küçük” ise (ör. dev model, büyük görsel) yine anlamlı olur.



## 9) Karar Rehberi (Cheat Sheet)

### SyncBatchNorm kullan ✅
- DDP (çoklu GPU / çoklu process) kullanıyorsun
- Per-GPU batch küçük (1–4–8 gibi)
- CNN tabanlı model (Conv ağırlıklı)
- Detection/Segmentation gibi BN hassas görevler

### SyncBatchNorm kullanma ❌ (çoğu durumda)
- Tek GPU / tek process
- Per-GPU batch zaten büyük (göreli olarak)
- Transformer ağırlıklı mimari (LN kullan)
- İletişim çok pahalı ve GN ile benzer kalite alabiliyorsun

### Alternatif öneriler
- Per-GPU batch küçük + SyncBN pahalı ⇒ **GroupNorm**
- Batch büyütmek istiyorsun ⇒ gradient accumulation (ama BN sorununu tam çözmez)
- Stabilite istiyorsun ⇒ LR schedule / warmup / EMA / uygun augment stratejileri



## 10) Mini Demonstrasyon: BN istatistiklerinin “batch küçülünce” sapması

Aşağıdaki örnekte tek GPU’da çalışırken bile şunu gösterebiliriz:
- Batch küçüldükçe (B=2 vs B=32) batch istatistikleri daha gürültülü olur.
- Bu doğrudan SyncBN’i çalıştırmak değil ama **neden** SyncBN’e ihtiyaç duyulduğunu sezdirir.


In [5]:

import torch
import torch.nn as nn

torch.manual_seed(0)

# Aynı layer, farklı batch boyutlarıyla çıkan batch mean/var örneği
bn = nn.BatchNorm2d(16, affine=False, track_running_stats=False)  # sadece batch stats görmek için
bn.train()

def stats_for_batch(B):
    x = torch.randn(B, 16, 32, 32)  # (B,C,H,W)
    with torch.no_grad():
        # BN forward içinde mean/var hesaplar; ama biz doğrudan kendimiz de görebiliriz:
        # Kanal bazlı, batch ve spatial üzerinde
        mu = x.mean(dim=(0,2,3))
        var = x.var(dim=(0,2,3), unbiased=False)
    return mu, var

mu2, var2 = stats_for_batch(2)
mu32, var32 = stats_for_batch(32)

print("B=2  mean (ilk 5 kanal):", mu2[:5])
print("B=32 mean (ilk 5 kanal):", mu32[:5])
print("B=2  var  (ilk 5 kanal):", var2[:5])
print("B=32 var  (ilk 5 kanal):", var32[:5])


B=2  mean (ilk 5 kanal): tensor([ 0.0049, -0.0272, -0.0208, -0.0453, -0.0016])
B=32 mean (ilk 5 kanal): tensor([ 0.0013, -0.0023, -0.0025,  0.0018, -0.0067])
B=2  var  (ilk 5 kanal): tensor([1.0341, 1.0294, 0.9736, 0.9417, 0.9718])
B=32 var  (ilk 5 kanal): tensor([0.9924, 1.0024, 1.0140, 0.9981, 0.9955])



**Yorum:**  
B=2 gibi küçük batch’te kanal başına mean/var daha “oynak” çıkar. Çoklu GPU’da her GPU B=2 görüyorsa, her GPU’nun BN’i farklı normalize eder ⇒ eğitim “dağılır”.  
SyncBN bu istatistikleri GPU’lar arasında toplayıp daha stabil hale getirir.

---

## 11) Pratik Checklist 
- [ ] Çoklu GPU mu? (DDP)  
- [ ] Per-GPU batch küçük mü?  
- [ ] CNN ağırlıklı model mi?  
- [ ] `convert_sync_batchnorm` DDP’den önce mi yapıldı?  
- [ ] `DistributedSampler` kullanıyor musun?  
- [ ] `model.train()` / `model.eval()` doğru mu?  
- [ ] Overhead’e değer mi? (mAP/IoU artışı vs hız kaybı)

---

## 12) Son Notlar (İleri Seviye İpuçları)
- Bazı modern framework’ler “Frozen BN” kullanır: BN’i eğitimde sabitler (özellikle finetune). Bu, küçük batch’te alternatif olabilir.
- Detection ekosisteminde GN/SyncBN seçimi çoğu zaman “altyapı + hız” ile belirlenir.
- Eğer backbone pretrain + küçük batch finetune yapıyorsan:  
  - BN katmanlarını freeze etmek veya GN’ye çevirmek bazen daha stabil çıkar.
